# Cleaning script

Este script carga el dataset completo, lo limpia y a partir de el construye el dataset de entrenamiento y validacion.

## Import libraries and data

In [1]:
import pandas as pd
import unicodedata
import re
from random import shuffle

In [2]:
df = pd.read_csv("Textos_griegos.csv")

In [3]:
df.head()

,Autor,Obra,Fragmento,Texto
0,Colluthus,Rape of Helen,1,νύμφαι Τρωιάδες. ποταμοῦ Ξάνθοιο γενέθλη. αἳ π...
1,Xenophon,Memorabilia,1.1.1,πολλάκις ἐθαύμασα τίσι ποτὲ λόγοις Ἀθηναίους ἔ...
2,Xenophon,Memorabilia,1.1.2,πρῶτον μὲν οὖν. ὡς οὐκ ἐνόμιζεν οὓς ἡ πόλις νο...
3,Xenophon,Memorabilia,1.1.3,ὁ δ᾽ οὐδὲν καινότερον εἰσέφερε τῶν ἄλλων. ὅσοι...
4,Xenophon,Memorabilia,1.1.4,ἀλλ᾽ οἱ μὲν πλεῖστοί φασιν ὑπό τε τῶν ὀρνίθων ...


## Basic cleaning

In [4]:
df.dropna(inplace=True)

In [5]:
autores_more_than = df.Autor.value_counts()[df.Autor.value_counts() > 10].index
df = df.loc[df.Autor.isin(autores_more_than)]

In [6]:
def sustituirchars(linea):
    return linea.replace('.',',').replace(',,','.')


In [7]:
df.Texto=df.Texto.apply(lambda x: sustituirchars(x))
df.Autor=df.Autor.apply(lambda x: sustituirchars(x))
df.Obra=df.Obra.apply(lambda x: sustituirchars(x))

In [8]:
def strip_accents_and_lowercase(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn').lower()

In [9]:
df.Texto=df.Texto.apply(lambda x: strip_accents_and_lowercase(x))

In [10]:
def cleanchars(t):
    # Eliminamos lo que haya entre corchetes y entre paréntesis
    a = re.sub(r'\[.+\]', ' ', str(t))
    a = re.sub(r'\(.+\)', ' ', str(a))
    # Quitar aquello que no sea griego o sean números
    a = re.sub(r'[A-Za-z0-9]', ' ', a)
    # Quitar puntos repetidos
    a = re.sub(r'\.{2,}', ' ', a)
    # Sustituir las contracciones por su forma no contraída
    a = re.sub(r'δ’', 'δε', a)
    a = re.sub(r'ουδʼ', 'ουδε', a)
    a = re.sub(r'αλλ’', 'αλλα', a)
    a = re.sub(r'υπʼ', 'υπο', a)
    # Sustituir caracteres extraños
    a = re.sub(r'[ʼ\᾿\*\|\»\«]', '', a)
    a = re.sub(r' [,\.] ', ' ', a)
    a = re.sub(r'^. ', ' ', a)
    # Quitar espacios extra en caso de haber
    a = re.sub(r'\s+', ' ', a, flags=re.I)
    return a.strip()

In [11]:
df.Texto = df.Texto.apply(lambda x: cleanchars(x))

## Splitting data

In [13]:
val_df_list = []
for autor in df.Autor.unique():
    books = len(df[df.Autor == autor].Obra.unique())
    if books > 10:
        obras = list(df[df.Autor==autor].Obra.unique())
        shuffle(obras)
        print(autor)
        print('\tObras en total: '+str(books))
        print('\tObras a separar: '+str(books//10))
        print('\tSiendo éstas: '+', '.join(obras[:books//10]))
        for obra in obras[:books//10]:
            val_df_list.append(df[df.Obra==obra])
            df.drop(df[df.Obra==obra].index, inplace=True)
val_df = pd.concat(val_df_list)
            

Eusebius of Caesarea
	Obras en total: 11
	Obras a separar: 1
	Siendo éstas: De Martyribus Palaestinae (Recensio Brevior)
Demosthenes
	Obras en total: 59
	Obras a separar: 5
	Siendo éstas: For the Megalopolitans, Against Evergus and Mnesibulus, Philippic 2, Against Onetor, Philip
Isocrates
	Obras en total: 29
	Obras a separar: 2
	Siendo éstas: Against Callimachus, To Philip
Aristophanes
	Obras en total: 11
	Obras a separar: 1
	Siendo éstas: Acharnians
Theophrastus
	Obras en total: 23
	Obras a separar: 2
	Siendo éstas: DeLapidibus, DeNervorumResolutione
Lysias
	Obras en total: 34
	Obras a separar: 3
	Siendo éstas: For the Soldier, Against Theomnestus 1, Against Diogeiton
Aristotle
	Obras en total: 38
	Obras a separar: 3
	Siendo éstas: deInterpretatione, Meteorologica, DePartibusAnimalium
Plutarch
	Obras en total: 18
	Obras a separar: 1
	Siendo éstas: Cleomenes
Euripides
	Obras en total: 14
	Obras a separar: 1
	Siendo éstas: Bacchae
Appian
	Obras en total: 14
	Obras a separar: 1
	Siendo é

In [14]:
val_df.describe()

,Autor,Obra,Fragmento,Texto
count,4292,4292,4292,4292
unique,13,34,1514,4168
top,Euripides,Bacchae,1,του μεν νομου διαρρηδην αγορευοντος τους εν τω...
freq,1204,1204,97,33


In [21]:
with open('val_df.csv', 'w', encoding = 'utf-8') as f:
    val_df.to_csv(f, sep='>', index=False, line_terminator = '\n', encoding='utf-8')

In [23]:
with open('training_df.csv', 'w', encoding='utf-8') as f:
    df.to_csv(f, sep='>', index=False, line_terminator = '\n', encoding='utf-8')